# For Collab

In [41]:
collab=1
if collab:
  from google.colab import drive
  drive.mount('/content/drive')
#prashantpath
  %cd /content/drive/MyDrive/Projects/NepSense/Prashant/Preprocessing

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Projects/NepSense/Prashant/Preprocessing


In [42]:
!ls

'Combined Dataset.ipynb'   Nepali_nlp   nepali-stemmer	'Preprocessing Datas'   preprocessing.ipynb


In [43]:
!ls "./Preprocessing Datas"

'Combined Manual Tagged Data_2024-06-06T01:03:07.xlsx'	'Emoji Sheets - Emoji Only.csv'
'Combined Manual Tagged Data Latest.xlsx'		 Outputs
'Combined Manual Tagged Data.xlsx'


# Installing required Library

In [44]:
# %pip install langdetect
# %pip install ipywidgets
# !git clone https://github.com/sushil79g/Nepali_nlp.git
# %pip install spello
# %pip install pytesseract
# %pip install gensim==4.3.2
# %pip install requests==2.31.0
# %pip install wget==3.2
# %pip install beautifulsoup4==4.12.3
# %pip install news-please==1.5.44
# %pip install pytesseract==0.3.10
# %pip install spello
# %pip install snowballstemmer==2.2.0
# %pip install tensorflow
# %pip install sentencepiece==0.2.0
# %pip install scikit-learn==1.4.2
# %pip install pydload==1.0.9
# %pip install tensorboardX==2.6.2.2
# %pip install fairseq
# %pip install opencv-python==4.9.0.80
# %pip install transformers==4.41.0

# Input and Output paths

In [45]:
combined_manaul_tagged_datas_path=f"./Preprocessing Datas/Combined Manual Tagged Data Latest.xlsx"
emoji_list_datas_path=f"./Preprocessing Datas/Emoji Sheets - Emoji Only.csv"

# Importing Datasets

In [46]:
import pandas as pd
combined_manual_tagged_df=pd.read_excel(combined_manaul_tagged_datas_path)
combined_manual_tagged_df

,Unnamed: 0,ai4bharat,Profanity,Profanity_Matched_Pattern,Offensive,Offensive_Matched_Pattern,Gender
0,0,भाक भोसडीके,1.0,भोसडी,1.0,भोसडी,1
1,1,पिक्चर स्पीक्स वाक मुजी,1.0,muji,1.0,muji,1
2,2,हे मुजी....,1.0,muji,1.0,muji,1
3,3,लस्तैइ ह्यान्डसम के मुजी 🤣❤️❤️❤️,1.0,muji,1.0,muji,1
4,4,आयो एमजी को कडा पर्सुस्तति♥️♥️♥️,1.0,mg,1.0,mg,1
...,...,...,...,...,...,...,...
16142,45,केपी बा इ लव योउ❤️,0.0,NaN,0.0,NaN,Unknown
16143,46,"नाटक,नौटंकी",0.0,NaN,0.0,NaN,1
16144,47,गुड लक बाबा,0.0,NaN,0.0,NaN,1
16145,48,नाटक पानी कति सम्मः गर्नः जेनी को बुडो ले चुना...,0.0,NaN,0.0,NaN,Unknown


In [47]:
emoji_df = pd.read_csv(emoji_list_datas_path)
emoji_df

,Emoji_List
0,U0000023
1,U000002A
2,U0000030
3,U0000031
4,U0000032
...,...
1421,U000E0062
1422,U000E0065
1423,U000E0067
1424,U000E006E


# Preprocessing

In [48]:
combined_manual_tagged_df["preprocessing_text"]=combined_manual_tagged_df["ai4bharat"].apply(lambda x: x.lower() if isinstance(x,str) else x)

### Remove emoji from text

In [49]:
import re
import pandas as pd
import tqdm

# Extract the emojis into a list
emoji_list = emoji_df['Emoji_List'].tolist()

# Start the pattern string
pattern = '['

# Append each code point to the pattern string, ensuring each one is 8 digits
for cp in emoji_list:
    pattern += f'\\U{cp[1:]:0>8}'

# Close the pattern string
pattern += ']'

# Compile the regular expression
emoji_pattern = re.compile(pattern, re.UNICODE)

# function to completely remove the emojis from the comments using re
def remove_emojis(text):
    if not isinstance(text, str):
        return text  # or return an empty string: return ''

    # Compile the regular expression
    emoji_pattern = re.compile(pattern, re.UNICODE)

    # Use the sub method to remove emojis
    text_no_emojis = emoji_pattern.sub(r'', text)
    return text_no_emojis

In [50]:
from tqdm import tqdm

tqdm.pandas()
combined_manual_tagged_df["preprocessing_text"]=combined_manual_tagged_df["preprocessing_text"].progress_apply(remove_emojis)

100%|██████████| 16147/16147 [00:02<00:00, 6805.98it/s]


### Stemming

In [51]:
from Nepali_nlp.Nepali_nlp import Stem
text = 'नेताहरुमा यत्तिको राजनैतिक संस्कार हुनुपर्छ'
Stem().rootify(text)

['नेता', 'यत्ति', 'राजनैतिक', 'संस्कार', 'हु']

In [52]:
def nepali_stemmer(text):
    """
    Stem Nepali text using the Stem class.

    Parameters:
    text (str): The input string to be stemmed.

    Returns:
    str: The stemmed string.
    """
    if not isinstance(text, str):
        return text  # or return an empty string: return ''

    temp_result = Stem().rootify(text)
    temp_result = " ".join(temp_result)
    return temp_result

In [53]:
combined_manual_tagged_df["preprocessing_text"] = combined_manual_tagged_df["preprocessing_text"].progress_apply(nepali_stemmer)
combined_manual_tagged_df

100%|██████████| 16147/16147 [00:06<00:00, 2416.74it/s]


,Unnamed: 0,ai4bharat,Profanity,Profanity_Matched_Pattern,Offensive,Offensive_Matched_Pattern,Gender,preprocessing_text
0,0,भाक भोसडीके,1.0,भोसडी,1.0,भोसडी,1,भाक भोसडीके
1,1,पिक्चर स्पीक्स वाक मुजी,1.0,muji,1.0,muji,1,पिक्चर स्पीक्स वाक मुजी
2,2,हे मुजी....,1.0,muji,1.0,muji,1,हे मुजी....
3,3,लस्तैइ ह्यान्डसम के मुजी 🤣❤️❤️❤️,1.0,muji,1.0,muji,1,लस्तैइ ह्यान्डसम के मुजी
4,4,आयो एमजी को कडा पर्सुस्तति♥️♥️♥️,1.0,mg,1.0,mg,1,आ एमजी कडा पर्सुस्तति
...,...,...,...,...,...,...,...,...
16142,45,केपी बा इ लव योउ❤️,0.0,NaN,0.0,NaN,Unknown,केपी बा इ लव योउ
16143,46,"नाटक,नौटंकी",0.0,NaN,0.0,NaN,1,"नाटक,नौटं"
16144,47,गुड लक बाबा,0.0,NaN,0.0,NaN,1,गुड लक बाबा
16145,48,नाटक पानी कति सम्मः गर्नः जेनी को बुडो ले चुना...,0.0,NaN,0.0,NaN,Unknown,नाटक पानी कति सम्मः गर्नः जेनी बुडो चुनाब बे...


### Stop Word Removal

In [54]:
import nltk
nltk.download('stopwords')
stopword=set(nltk.corpus.stopwords.words('nepali'))
stopword

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


{'अक्सर',
 'अगाडी',
 'अझै',
 'अनुसार',
 'अन्तर्गत',
 'अन्य',
 'अन्यत्र',
 'अन्यथा',
 'अब',
 'अरु',
 'अरुलाई',
 'अर्को',
 'अर्थात',
 'अर्थात्',
 'अलग',
 'आए',
 'आजको',
 'आत्म',
 'आदि',
 'आफू',
 'आफूलाई',
 'आफ्नै',
 'आफ्नो',
 'आयो',
 'उदाहरण',
 'उनको',
 'उनले',
 'उप',
 'उहालाई',
 'एउटै',
 'एक',
 'एकदम',
 'ओठ',
 'औं',
 'कतै',
 'कम से कम',
 'कसरी',
 'कसै',
 'कसैले',
 'कहाँबाट',
 'कहिलेकाहीं',
 'का',
 'कि',
 'किन',
 'किनभने',
 'कुनै',
 'कुरा',
 'कृपया',
 'के',
 'केही',
 'को',
 'कोही',
 'क्रमशः',
 'गए',
 'गयौ',
 'गरि',
 'गरी',
 'गरेका',
 'गरेको',
 'गरेर',
 'गरौं',
 'गर्छ',
 'गर्छु',
 'गर्दै',
 'गर्न',
 'गर्नु',
 'गर्नुपर्छ',
 'गर्ने',
 'गैर',
 'चार',
 'चाले',
 'चाहनुहुन्छ',
 'चाहन्छु',
 'चाहिए',
 'छ',
 'छन्',
 'छु',
 'छू',
 'छैन',
 'छौं',
 'जताततै',
 'जब',
 'जबकि',
 'जसको',
 'जसबाट',
 'जसमा',
 'जसलाई',
 'जसले',
 'जस्तै',
 'जस्तो',
 'जस्तोसुकै',
 'जहाँ',
 'जान',
 'जाहिर',
 'जुन',
 'जे',
 'जो',
 'ठीक',
 'त',
 'तत्काल',
 'तथा',
 'तदनुसार',
 'तपाई',
 'तपाईको',
 'तर',
 'तल',
 'तापनी',
 'तिनिहरुला

In [55]:
import pandas as pd

def remove_stop_words(text, stop_words):
    text = str(text)  # Convert to string
    words = text.split()
    filtered_words = [word for word in words if word not in stop_words]
    return ' '.join(filtered_words)

In [56]:
import numpy as np

# Check for missing values and replace them with an empty string
combined_manual_tagged_df["preprocessing_text"].fillna("", inplace=True)

# Apply the remove_stop_words function
combined_manual_tagged_df["preprocessing_text"] = combined_manual_tagged_df["preprocessing_text"].progress_apply(lambda x: remove_stop_words(x, stopword))


100%|██████████| 16147/16147 [00:00<00:00, 199651.04it/s]


### Removing Punctuation


In [57]:
# Sample DataFrame containing punctuation information
punctuation_data=["।", ",", ";", "?", "!", "(", "” “", "—", "–", ":", "…", "'", "()", "&",".","-", " 🏻","ई🏿🏻"," 🏿🏿"," 🏿🏿🏿🏿🏿🏿🏿🏿🏿🏿"]

punctuation_df = pd.DataFrame(punctuation_data)

def remove_punctuation(text):
    """
    Remove specified punctuation marks from the text.

    Parameters:
    text (str): The input string from which punctuation will be removed.

    Returns:
    str: The string with punctuation removed.
    """
    if not isinstance(text, str):
        return text  # or return an empty string: return ''

    for punct in punctuation_df[0]:
        text = text.replace(punct, "")
    return text


In [58]:
combined_manual_tagged_df["preprocessing_text"] = combined_manual_tagged_df["preprocessing_text"].progress_apply(remove_punctuation)

100%|██████████| 16147/16147 [00:00<00:00, 91829.34it/s]


### Only take unique values

In [59]:
# import pandas as pd

# # Sample DataFrame with duplicate values
# data = {'A': [1, 2, 6, 3, 4],
#         'B': [5, 6, 6, 7, 8],
#         'C': [9, 10, 13, 11, 12]}

# df = pd.DataFrame(data)

# # Remove duplicate values from column 'A'
# df = df.drop_duplicates(subset=['A'])

# print(df)
combined_manual_tagged_df = combined_manual_tagged_df.drop_duplicates(subset=['preprocessing_text'])

### Remove the word if whitespace isnot found after 10 letters

In [60]:
# !pip install regex
import regex as re

def remove_long_words_without_whitespace(text):
    # Define a regular expression pattern to find words longer than 10 characters in any language
    pattern = r'\b\w{11,}\b'

    # Use re.sub to replace the matched words with an empty string
    modified_text = re.sub(pattern, '', text)

    # Check if the text was modified
    if modified_text != text:
        return ''  # Return empty string if any words were removed
    else:
        return text  # Return the original text if no changes were made

# Test the function with a text containing long words in Devanagari script
result = remove_long_words_without_whitespace("जस्टिसफरनि जस्")
print(result)


जस्टिसफरनि जस्


In [61]:
combined_manual_tagged_df["preprocessing_text"] = combined_manual_tagged_df["preprocessing_text"].progress_apply(remove_long_words_without_whitespace)

100%|██████████| 11759/11759 [00:00<00:00, 85445.15it/s]
<ipython-input-61-cb851c8d0da4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_manual_tagged_df["preprocessing_text"] = combined_manual_tagged_df["preprocessing_text"].progress_apply(remove_long_words_without_whitespace)


### Removing Null row and having empty string

In [62]:
# Count occurrences where 'preprocessing_text' is an empty string, null, or NaN
empty_or_null_count = combined_manual_tagged_df["preprocessing_text"].isna().sum() + (combined_manual_tagged_df["preprocessing_text"] == "").sum()

# Print the result
print(f"The number of occurrences where 'preprocessing_text' is an empty string, null, or NaN is: {empty_or_null_count}")

The number of occurrences where 'preprocessing_text' is an empty string, null, or NaN is: 1105


In [63]:
# Remove rows where 'preprocessing_text' is null or NaN
combined_manual_tagged_df = combined_manual_tagged_df.dropna(subset=["preprocessing_text"])

# Remove rows where 'preprocessing_text' is an empty string
combined_manual_tagged_df = combined_manual_tagged_df[combined_manual_tagged_df["preprocessing_text"] != ""]

# Saving the Preprocessed file

In [64]:
count_profanity = combined_manual_tagged_df[combined_manual_tagged_df["Profanity"] == 1].shape[0]
print("The count of rows where the value in the 'Profanity' column is 1:", count_profanity)

The count of rows where the value in the 'Profanity' column is 1: 2052


In [65]:
count_offensive = combined_manual_tagged_df[combined_manual_tagged_df["Offensive"] == 1].shape[0]
print("The count of rows where the value in the 'Offensive' column is 1:", count_offensive)

The count of rows where the value in the 'Offensive' column is 1: 3078


In [66]:
combined_manual_tagged_df

,Unnamed: 0,ai4bharat,Profanity,Profanity_Matched_Pattern,Offensive,Offensive_Matched_Pattern,Gender,preprocessing_text
0,0,भाक भोसडीके,1.0,भोसडी,1.0,भोसडी,1,भाक भोसडीके
1,1,पिक्चर स्पीक्स वाक मुजी,1.0,muji,1.0,muji,1,पिक्चर स्पीक्स वाक मुजी
2,2,हे मुजी....,1.0,muji,1.0,muji,1,हे मुजी
3,3,लस्तैइ ह्यान्डसम के मुजी 🤣❤️❤️❤️,1.0,muji,1.0,muji,1,लस्तैइ ह्यान्डसम मुजी
4,4,आयो एमजी को कडा पर्सुस्तति♥️♥️♥️,1.0,mg,1.0,mg,1,आ एमजी कडा पर्सुस्तति
...,...,...,...,...,...,...,...,...
16141,44,म पनि फोनको प्रतीक्षा मै छु एकदिन हजुरको फोन अ...,0.0,NaN,0.0,NaN,Unknown,फोन प्रतीक्षा एकदिन हजुर फोन अवस्य आउ
16143,46,"नाटक,नौटंकी",0.0,NaN,0.0,NaN,1,नाटकनौटं
16144,47,गुड लक बाबा,0.0,NaN,0.0,NaN,1,गुड लक बाबा
16145,48,नाटक पानी कति सम्मः गर्नः जेनी को बुडो ले चुना...,0.0,NaN,0.0,NaN,Unknown,नाटक पानी कति सम्मः गर्नः जेनी बुडो चुनाब बेला...


In [67]:
import os
excel_file_path = './Preprocessing Datas/Outputs/preprocessed_data.xlsx'
os.makedirs(os.path.dirname(excel_file_path), exist_ok=True)
combined_manual_tagged_df.to_excel(excel_file_path,index=False)
# Print a success message
print(f"File saved successfully (path: {excel_file_path})")

File saved successfully (path: ./Preprocessing Datas/Outputs/preprocessed_data.xlsx)


In [68]:
pwd

'/content/drive/MyDrive/Projects/NepSense/Prashant/Preprocessing'